# Movie Review Sentiment Prediction

This report is written with jupyter notebook and converted to pdf, so if you have jupyter installed, you can run the file report.ipynb.

Use nltk to tokenize and count the number of each words. For information on installation see README.

In [165]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import Counter
from nltk.tokenize import RegexpTokenizer

### Preprocessing

Use os.listdir to find all file names and then iterate throught them and read each file into a string, omitting linebreaks and apostrophes.

In [167]:
reviews_dir_pos = 'review_polarity/txt_sentoken/pos'
reviews_dir_neg = 'review_polarity/txt_sentoken/neg'
pos_reviews = os.listdir(reviews_dir_pos)
neg_reviews = os.listdir(reviews_dir_neg)

positive_str = []
negative_str = []
# read in positive reviews
for review in pos_reviews:
    with open(os.path.join(reviews_dir_pos, review), 'r') as file:
        review_str = file.read().replace('\n', '').replace("'", '')
        positive_str.append(review_str)
        
for review in neg_reviews:
    with open(os.path.join(reviews_dir_neg, review), 'r') as file:
        review_str = file.read().replace('\n', '')
        negative_str.append(review_str)

Get a list of all words


In [191]:
import pprint

mega_str = ''.join(positive_str + negative_str)
tokenizer = RegexpTokenizer(r'\w+')
all_count = Counter(tokenizer.tokenize(mega_str))

pp = pprint.PrettyPrinter(width=80, compact=True)
print('The most common {} words in all reviews are: '.format(60))
top_common = all_count.most_common(60)
pp.pprint(top_common)

# delete top 60 common words
for key in list(zip(*top_common))[0]:
    del all_count[key]

print('============================================')
print('Total number of unique words left in the word set: {}'.format(len(all_count)))

feature_keys = list(dict(all_count).keys())

The most common 60 words in all reviews are: 
[('the', 76528), ('a', 38100), ('and', 35576), ('of', 34123), ('to', 31937),
 ('is', 25195), ('in', 21822), ('that', 15566), ('it', 14200), ('as', 11378),
 ('with', 10792), ('for', 9961), ('his', 9587), ('this', 9577), ('film', 9196),
 ('s', 9077), ('but', 8634), ('he', 8267), ('i', 8259), ('on', 7382),
 ('are', 6949), ('by', 6261), ('be', 6173), ('one', 5816), ('an', 5744),
 ('movie', 5665), ('not', 5577), ('who', 5548), ('from', 4999), ('at', 4986),
 ('you', 4943), ('was', 4940), ('have', 4901), ('has', 4719), ('they', 4614),
 ('her', 4522), ('all', 4373), ('its', 4163), ('like', 3690), ('so', 3683),
 ('out', 3637), ('t', 3628), ('about', 3522), ('up', 3405), ('there', 3364),
 ('more', 3347), ('when', 3258), ('what', 3233), ('which', 3161), ('or', 3146),
 ('their', 3122), ('some', 2985), ('she', 2941), ('just', 2905), ('if', 2799),
 ('him', 2633), ('into', 2623), ('can', 2607), ('we', 2595), ('even', 2565)]
Total number of unique words le

At this stage, we've got our feature list.


In [327]:
pos_tokens = []
neg_tokens = []

# for each review find and delete top 10 common words
for review in positive_str:
    count = dict(Counter(tokenizer.tokenize(review)))
    feature_dict = {}
    for key in feature_keys:
        if key in count:
            feature_dict[key] = count[key]
        else:
            feature_dict[key] = 0
    pos_tokens.append(feature_dict)
    
for review in negative_str:
    count = dict(Counter(tokenizer.tokenize(review)))
    for key in feature_keys:
        if key in count:
            feature_dict[key] = count[key]
        else:
            feature_dict[key] = 0
    neg_tokens.append(feature_dict)

The final step: shuffle the data.

In [328]:
for i in range(0, len(pos_tokens)):
    pos_tokens[i]['@'] = 1
    
for i in range(0, len(neg_tokens)):
    neg_tokens[i]['@'] = -1

The final step: shuffle the data.

In [330]:
from random import shuffle

features = pos_tokens + neg_tokens
print('Total number of reviews: {}, each has {} features'.format(len(features),
                                                                 len(features[0])-1))

shuffle(features)

In [330]:
labels = []
for i in range(0, len(features)):
    labels.append(features[i]['@'])

for entry in features:
    try:
        del entry['@']
    except KeyError:
        pass


### Training the perceptron

In [351]:
from learner.perceptron import Perceptron

features_list = [list(entry.values()) for entry in features]

AttributeError: 'list' object has no attribute 'shape'

In [352]:
clf = Perceptron()
print(clf.train(features_list, labels))

AttributeError: 'list' object has no attribute 'shape'